In [47]:
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../src'))

from scipy.signal import ShortTimeFFT
from scipy.signal import detrend

from scipy.signal.windows import gaussian
import data_loader
import datetime
import numpy as np
import pandas as pd

from cdflib.epochs_astropy import CDFAstropy as cdfepoch
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.colors

import importlib
importlib.reload(data_loader)

%matplotlib qt

In [48]:
start_time = datetime.datetime(year = 2016, month = 1, day = 1, hour = 1, minute = 1, second = 1)
end_time = datetime.datetime(year = 2016, month = 4, day = 1, hour = 1, minute = 1, second = 1)

In [51]:
OMNI = data_loader.load_raw_data_from_config(id = ["OMNI", "ONE_MIN_RESOLUTION"],
                                             start = start_time,
                                             end = end_time)

time = cdfepoch.unixtime(OMNI["Epoch"])
Bz = OMNI["BZ_GSM"]
density = OMNI["proton_density"]
Vz = OMNI["Vz"]

within_time_range = (start_time.timestamp() < time) & (time < end_time.timestamp())
valid_alfven_velocity_parameters = (density < 990) & (Bz < 100) & (Vz < 100)

time = time[within_time_range & valid_alfven_velocity_parameters]
Bz = Bz[within_time_range & valid_alfven_velocity_parameters]
density = density[within_time_range & valid_alfven_velocity_parameters]
Vz = Vz[within_time_range & valid_alfven_velocity_parameters]


Bz = np.interp(np.linspace(time[0], time[-1], int((time[-1] - time[0])/60)), time, Bz)
density = np.interp(np.linspace(time[0], time[-1], int((time[-1] - time[0])/60)), time, density)
Vz = np.interp(np.linspace(time[0], time[-1], int((time[-1] - time[0])/60)), time, Vz)

Vz_detrended = pd.Series(detrend(Vz))
VAz_detrended = pd.Series(detrend(Bz / np.sqrt(density)))

In [63]:
plt.plot(np.clip(Vz_detrended.rolling(1440, center=True).corr(VAz_detrended, "spearman"), 0, 1.0))

In [ ]:
g_std = 60
w = gaussian(60, std=g_std, sym=True)

SFT = ShortTimeFFT(w, hop=1, fs=1, scale_to='psd')

Sx2 = SFT.spectrogram(x = Bz / np.sqrt(density), detr="constant")

In [17]:
fig, ax = plt.subplots(1, 1, sharex=True)

image = ax.imshow(10 * np.log10(np.fmax(Sx2, 1e-4)),
                     aspect="auto",
                     extent=SFT.extent(len(Bz)),
                     norm=matplotlib.colors.Normalize(vmin=-40, vmax=20))


ax.set_title(rf"Spectrogram ({SFT.m_num*SFT.T:g}$\,s$ Gaussian " +
              rf"window, $\sigma_t={g_std*SFT.T:g}\,$s)")
ax.set(xlabel=f"Time $t$ in seconds ({SFT.p_num(len(Bz))} slices, " +
               rf"$\Delta t = {SFT.delta_t:g}\,$s)",
        ylabel=f"Freq. $f$ in Hz ({SFT.f_pts} bins, " +
               rf"$\Delta f = {SFT.delta_f:g}\,$Hz)")

axins = inset_axes(
        ax,
        width="1%",  # width: 5% of parent_bbox width
        height="100%",  # height: 50%
        loc="lower left",
        bbox_to_anchor=(1.01, 0, 1, 1),
        bbox_transform=ax.transAxes,
        borderpad=0
    )

plt.colorbar(image, cax=axins, label='Power Spectral Density ' + r"$20\,\log_{10}|S_x(t, f)|$ in dB")